# Week 5 - Natural Language Processing (NLP)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
import src.week5_func as wk5
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# data processed and saved with week5_func.save_as_pkl()
df_raw = pd.read_pickle('./data/df_raw.pkl')
df_raw_test = pd.read_pickle('./data/df_raw_test.pkl')

In [3]:
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

In [4]:
# lets start by predicting just positive stuff.
df_pos = df_raw[df_raw['positive']==1]
str_all=[]
def func(text):
    str_all.append(text)
df_pos.apply(lambda x: func(x['text']),axis=1)

# learn on the first 10 smaples
strrring = ''
for i in range(10):
    strrring+=str_all[i]

print(strrring)
    

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to 

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(strrring))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in strrring])

In [6]:
(encoded.shape)
int2char

{0: 'f',
 1: 'C',
 2: '.',
 3: 'I',
 4: '*',
 5: '7',
 6: "'",
 7: 'r',
 8: 'b',
 9: 'i',
 10: 'A',
 11: 'J',
 12: 'V',
 13: 'S',
 14: 'a',
 15: 'u',
 16: 'T',
 17: 'q',
 18: '/',
 19: 'Y',
 20: '6',
 21: 'n',
 22: 'B',
 23: 'v',
 24: 's',
 25: 'p',
 26: 'd',
 27: '2',
 28: '~',
 29: 'M',
 30: ')',
 31: '(',
 32: 'R',
 33: 'H',
 34: '-',
 35: 'z',
 36: 'W',
 37: 'c',
 38: '5',
 39: 'N',
 40: 'U',
 41: 'G',
 42: ' ',
 43: '?',
 44: 'K',
 45: '!',
 46: 'E',
 47: 'D',
 48: 'O',
 49: 'e',
 50: 'j',
 51: 'w',
 52: 'P',
 53: '"',
 54: '\x85',
 55: 'h',
 56: 'm',
 57: '>',
 58: 'k',
 59: 'x',
 60: 'g',
 61: ',',
 62: ';',
 63: 'F',
 64: '0',
 65: 't',
 66: 'y',
 67: ':',
 68: 'o',
 69: '8',
 70: '1',
 71: 'l',
 72: 'L',
 73: '3',
 74: '<'}

In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros([len(arr), n_labels], dtype=np.float32)
    
    # Fill the appropriate elements with ones
#     one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
#     print(arr.flatten().tolist())
    one_hot[np.arange(one_hot.shape[0]), arr.flatten().int()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = torch.from_numpy(one_hot.reshape((*arr.shape, n_labels)))
    
    return one_hot
# print(type(line_tensor))

In [8]:
# arr = one_hot_encode(encoded, len(chars))
# np.shape(arr)

In [9]:
def get_batches(arr, seq_length): 
    X=[]
    y=[]
    for ind in range(np.shape(arr)[0]-seq_length):
        X.append(arr[ind:ind+seq_length]) #,:])
        y.append(arr[ind+seq_length])# ,:])
#     return np.moveaxis(np.array(X),0,1),np.array(y)
    return torch.from_numpy(np.array(X).reshape(-1,seq_length)).float(), torch.from_numpy(np.array(y)).float()
X,y = get_batches(encoded,10) #try without onehot.
# y_cold = y.max(1)[1]
print(np.shape(X),np.shape(y))

torch.Size([13606, 10]) torch.Size([13606])


In [10]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) #(129,128)
        self.i2o = nn.Linear(input_size + hidden_size, output_size) #(129,75)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, inputs, hidden):
#         print('forward sizes',input.size(),hidden.size())
        combined = torch.cat((inputs,hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# n_hidden = 128
# n_letters = 1 #len(chars)*10
# n_categories = len(chars)
# rnn = RNN(n_letters, n_hidden, n_categories)

# hidden =torch.zeros(1, n_hidden)
# output, next_hidden = rnn.forward(X[0,0].reshape(1,1), hidden)
# print(output,next_hidden)

In [11]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return int2char[category_i], category_i

# print(categoryFromOutput(output))

In [12]:
print(X.size(),y.size())

torch.Size([13606, 10]) torch.Size([13606])


In [13]:
import random

def randomChoice(l, t):
    randind = random.randint(0,np.shape(l)[0]-1)
    return l[randind ,:], t[randind] #, tc[randind]
def randomTrainingExample():
    randX, randy = randomChoice(X,y) #,y_cold)
#     print(randX)
#     reshaped_str = np.reshape(randX,(10,-1),order='C')
    trainstr = ''
    train = [int2char[ii.item()] for ii in randX]
    for ii in train: trainstr+=ii
#     print('[',trainstr,'][',int2char[randy.item()],']')
    return int2char[randy.item()], trainstr, randy, randX
#     return trainstr, int2char[np.argmax(randy).item()], randX.reshape(1,-1), randy, randy_c
randomTrainingExample()

(' ',
 'the latest',
 tensor(42.),
 tensor([65., 55., 49., 42., 71., 14., 65., 49., 24., 65.]))

In [14]:
criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)

In [15]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
#     print(np.shape(line_tensor))
    outputs = []
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i].reshape(1,-1), hidden)
        outputs.append(output)
    outputs = torch.stack(outputs)
    
    loss = criterion(output, category_tensor.reshape(1,).long()) #just use the final output to calculate the loss
#     print('lossis', output, category_tensor)
#     print(torch.min(torch.max(outputs,1)[1].reshape(-1,1).float()),torch.min(torch.max(category_tensor, 1)[1]))
#     loss = criterion(torch.max(outputs,1)[1].reshape(-1,1).float(), category_tensor)
    loss.backward(retain_graph=True)

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(learning_rate, p.grad.data)
#         print('post zero', p.grad.data)
    return output, loss.item()

In [16]:
import time
import math

n_iters = 5000
print_every = 100
plot_every = 100
learning_rate = 0.005

# initiate model
n_hidden = 32
n_letters = 75 #len(chars)*10
n_categories = len(chars)
rnn = RNN(n_letters, n_hidden, n_categories)

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()
# output, loss = train(y_cold, X)
# print(timeSince(start))
for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    line_tensor = one_hot_encode(line_tensor, n_categories)
#     print(line_tensor)
    output, loss = train(category_tensor.reshape(1,1), line_tensor)
#     output, loss = train(y_cold, X)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

100 2% (0m 0s) 4.3488 e associat / l ✗ (e)
200 4% (0m 0s) 4.3887 mputer, an / V ✗ (d)
300 6% (0m 0s) 4.3522  was follo / < ✗ (w)
400 8% (0m 0s) 4.5511 s much les / ! ✗ (s)
500 10% (0m 0s) 4.6264 ence. Agai / R ✗ (n)
600 12% (0m 1s) 5.1104 s. Jeffrey / 7 ✗ ( )
700 14% (0m 1s) 4.4187 her the "h / V ✗ (i)
800 16% (0m 1s) 5.0676 son deserv / V ✗ (e)
900 18% (0m 1s) nan teaching p / < ✗ (r)
1000 20% (0m 1s) nan  lovable i / < ✗ (n)
1100 22% (0m 2s) nan g more bui / < ✗ (l)
1200 24% (0m 2s) nan  pettiness / < ✗ ( )
1300 26% (0m 2s) nan ble as a c / < ✗ (o)
1400 28% (0m 2s) nan not used t / < ✗ (o)
1500 30% (0m 3s) nan anticipate / < ✗ (.)
1600 32% (0m 3s) nan such a fan / < ✗ (t)
1700 34% (0m 3s) nan <br /><br  / < ✗ (/)
1800 36% (0m 3s) nan tilt like  / < ✗ (a)
1900 38% (0m 3s) nan ime watch  / < ✗ (i)
2000 40% (0m 3s) nan  was the b / < ✗ (e)
2100 42% (0m 4s) nan Night List / < ✗ (e)
2200 44% (0m 4s) nan I'm here t / < ✗ (o)
2300 46% (0m 4s) nan ly, they c / < ✗ (o)
2400 48% (0m 4s) nan le

## Try LSTM

In [17]:
train

<function __main__.train(category_tensor, line_tensor)>

In [18]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
#         embeds = self.word_embeddings(sentence)
#         print(sentence.size())
        lstm_out, self.hidden = self.lstm(sentence, self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    
def train(category_tensor, line_tensor):
    hidden = lstm.init_hidden()
    lstm.zero_grad()
#     print(line_tensor.size())
#     outputs = []
    for i in range(line_tensor.size()[0]): # 10 chars per input
        output = lstm(line_tensor[i].view(1,1,-1))
#         outputs.append(output)
#     outputs = torch.stack(outputs)
    
    loss = criterion(output, category_tensor.reshape(1,).long()) #just use the final output to calculate the loss
#     print('lossis', output, category_tensor)
#     print(torch.min(torch.max(outputs,1)[1].reshape(-1,1).float()),torch.min(torch.max(category_tensor, 1)[1]))
#     loss = criterion(torch.max(outputs,1)[1].reshape(-1,1).float(), category_tensor)
    loss.backward(retain_graph=True)
    optimizer.step()

    # Add parameters' gradients to their values, multiplied by learning rate
#     for p in lstm.parameters():
#         p.data.add_(learning_rate, p.grad.data)
#         print('post zero', p.grad.data)
    return output, loss.item()

In [19]:
import time
EMBEDDING_DIM = 75
HIDDEN_DIM = 128
lstm = LSTM(EMBEDDING_DIM, HIDDEN_DIM, len(chars), len(chars))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm.parameters(), lr=0.1)
start = time.time()
n_iters = 5000
print_every = 100
plot_every = 100
for iter in range(1, n_iters + 1):
    lstm.zero_grad()
    category, line, category_tensor, line_tensor = randomTrainingExample()
    line_tensor = one_hot_encode(line_tensor, n_categories)
#     print(line_tensor.size())
    output, loss = train(category_tensor.reshape(1,1), line_tensor)
#     output, loss = train(y_cold, X)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

100 2% (0m 24s) 2.8514  well revi /   ✗ (e)
200 4% (1m 25s) 4.5284 . When tro / e ✗ (u)
300 6% (3m 1s) 2.1887 ncluding a /   ✓
400 8% (5m 12s) 1.8178 eet that w / e ✓
500 10% (9m 11s) 1.6378 a slighter /   ✓
600 12% (13m 35s) 2.8253  in the te /   ✗ (a)
700 14% (18m 34s) 4.3610 oo as a ma /   ✗ (d)
800 16% (24m 42s) 2.0495 ehouse are /   ✓
900 18% (31m 41s) 3.3503 ies; if Bo /   ✗ (l)
1000 20% (39m 21s) 0.9397 shed out a /   ✓
1100 22% (47m 22s) 1.5999  given the /   ✓
1200 24% (56m 22s) 3.4545 e named Do /   ✗ (n)
1300 26% (66m 3s) 1.5400 zing Saddl / e ✓
1400 28% (76m 28s) 2.8553 former lov /   ✗ (e)
1500 30% (87m 34s) 2.4109 film's nov /   ✗ (i)
1600 32% (99m 33s) 2.8081 the pettin /   ✗ (e)
1700 34% (112m 11s) 2.9091 Gabriel fi /   ✗ (n)
1800 36% (125m 14s) 1.7551 .<br /><br /   ✓
1900 38% (139m 2s) 3.7727  project o /   ✗ (f)
2000 40% (154m 9s) 2.7861 ure from t / e ✗ (h)
2100 42% (169m 59s) 2.9718  of suspen /   ✗ (s)
2200 44% (186m 44s) 2.8129 an art...  / n ✗ (t)
2300 46% (203m

In [52]:
def sample_from_probabilities(probabilities, topn=75):
    """Roll the dice to produce a random integer in the [0..ALPHASIZE] range,
    according to the provided probabilities. If topn is specified, only the
    topn highest probabilities are taken into account.
    :param probabilities: a list of size ALPHASIZE with individual probabilities
    :param topn: the number of highest probabilities to consider. Defaults to all of them.
    :return: a random integer
    """
    p = np.squeeze(np.exp(probabilities.detach().numpy()))
    p[np.argsort(p)[:-topn]] = 0
    p = p / np.sum(p)
    return np.random.choice(75, 1, p=p)[0]

line = 'hello how are y'

for ii in range(1000):
    linenum = [char2int[jj] for jj in line]
    inline = one_hot_encode(torch.from_numpy(np.array(linenum[-1:])),75).view(1,1,-1)
#     print(nn.Softmax(lstm.forward(inline)))
    line += int2char[sample_from_probabilities(lstm.forward(inline))][0]

print(line)

hello how are yli w ak rals fosueefhes< aaas Keloos hrl ad n hs 0uv thol a oi aus yov ariatire os fva, "fr Nace awre 0y lie as sodinh sad isiti.sdcs on oth seecs brocag sus ass uyia eat salco atolisg h, sias bqnu te a s cs hgt icn uariloArest /f: <. n/l voesestos tuns Osontho g ac >onp sol wi weny "atTesle lerG fa(tor fsod safans il *y liltlnafelv foslancs iti boochntopoilnn ootoa K. cuo) a atoh fevnd 6v.d Wochov z t?ith cy wicce iolles Wee ni as tins owonld Slc T wrcaoseterilon a pe nebaticil a ilis u1t gh   w- osg ,, wivv  hy atecla laneecetewc. s ows am)s innn fo hh eucd al ihe iis Fs n ear" soP ith s s?el fo dhole inmeo osifgpino ris Aonvenhavianvegl not fs r sn s n5ainre tf s, aneanes. nn ho th sss <s ssBo5 alne to :so.e as aolede)se nale an sirsis asah An sle m. uh k b Steh hh abnls fr tgl ulogcotloh. cns)r(soual Cs arls bssalion wmad caknt asilelth hocytg a aee, sonfre so0 " otit s oln wis Bo a-sy aldelan win s no njov asoel g 'eho f o s 7oyesg toemW s/uon ind it ia'a na ia sos

In [53]:
state_dict = lstm.state_dict()
np.save('state_dict_read_one.npy', state_dict) 